In [2]:
try:
    %load_ext google.colab.data_table
except Exception as e:
    pass
import json
import pandas as pd
from matplotlib import pyplot as plt
from time import sleep
try:
    %matplotlib inline
except Exception as e:
    pass


In [3]:
try:
    from google.colab import files
    uploaded = files.upload()
    data:dict = json.loads(uploaded['data-trunc.json'])
except Exception as e:
    import util
    with open('./data-trunc2.json') as fp:
        # 500ms~
        # data:dict = util.partial_json(fp,objs=26)
        data:dict = json.load(fp)

In [4]:
data

{'1577836803078': {'assetA': {'ask': 7189, 'bid': 7188.89},
  'assetB': {'ask': 7170.25, 'bid': 7169.5}},
 '1577836806371': {'assetA': {'ask': 7190.09, 'bid': 7190.05},
  'assetB': {'ask': 7170.25, 'bid': 7169.5}},
 '1577836806459': {'assetA': {'ask': 7190.09, 'bid': 7190.05},
  'assetB': {'ask': 7170.5, 'bid': 7169.5}},
 '1577836806474': {'assetA': {'ask': 7190.43, 'bid': 7190.05},
  'assetB': {'ask': 7170.5, 'bid': 7169.5}},
 '1577836806575': {'assetA': {'ask': 7190.39, 'bid': 7190.05},
  'assetB': {'ask': 7170.5, 'bid': 7169.5}},
 '1577836807242': {'assetA': {'ask': 7190.33, 'bid': 7189.92},
  'assetB': {'ask': 7170.5, 'bid': 7169.5}},
 '1577836807344': {'assetA': {'ask': 7190.25, 'bid': 7189.92},
  'assetB': {'ask': 7170.5, 'bid': 7169.5}},
 '1577836807444': {'assetA': {'ask': 7190.21, 'bid': 7189.92},
  'assetB': {'ask': 7170.5, 'bid': 7169.5}},
 '1577836807545': {'assetA': {'ask': 7190.12, 'bid': 7189.92},
  'assetB': {'ask': 7170.5, 'bid': 7169.5}},
 '1577836807649': {'assetA': 

In [5]:
# df = pd.read_json('./data-trunc.json', orient='index')
df = pd.DataFrame.from_dict(data,orient='index')

df

,assetA,assetB
1577836803078,"{'ask': 7189, 'bid': 7188.89}","{'ask': 7170.25, 'bid': 7169.5}"
1577836806371,"{'ask': 7190.09, 'bid': 7190.05}","{'ask': 7170.25, 'bid': 7169.5}"
1577836806459,"{'ask': 7190.09, 'bid': 7190.05}","{'ask': 7170.5, 'bid': 7169.5}"
1577836806474,"{'ask': 7190.43, 'bid': 7190.05}","{'ask': 7170.5, 'bid': 7169.5}"
1577836806575,"{'ask': 7190.39, 'bid': 7190.05}","{'ask': 7170.5, 'bid': 7169.5}"
...,...,...
1577837711136,"{'ask': 7178.32, 'bid': 7178.31}","{'ask': 7162.75, 'bid': 7159}"
1577837711173,"{'ask': 7178.32, 'bid': 7178.31}","{'ask': 7162.75, 'bid': 7162}"
1577837712988,"{'ask': 7178.32, 'bid': 7178.31}","{'ask': 7162.75, 'bid': 7161}"
1577837713474,"{'ask': 7178.32, 'bid': 7178.18}","{'ask': 7162.75, 'bid': 7161}"


In [6]:
def asks(state):
    ask_ = state['ask']
    return ask_
def bids(state):
    bid_ = state['bid']
    return bid_

In [7]:
def plot(obj, title=None):
    plt.figure(figsize=(15,7))
    plt.plot(obj)
    if title:
        plt.title(title, color='white')
    axes = plt.axes()
    axes.patch.set_facecolor('black')
    plt.xticks(rotation=45,color='white')
    plt.yticks(color='white')
    plt.show()

def plotmany(x, objs, title=None):
    plt.figure(figsize=(15,7))
    if title:
        plt.title(title, color='white')
    for obj, kwargs in objs:
        plt.plot(x, obj, **kwargs)
    plt.legend(loc='upper left')
    axes = plt.axes()
    axes.patch.set_facecolor('black')
    plt.xticks(rotation=45,color='white')
    plt.yticks(color='white')
    plt.show()

In [8]:
assetA:pd.DataFrame = df.assetA.apply([asks, bids])

,asks,bids
1577836803078,7189.00,7188.89
1577836806371,7190.09,7190.05
1577836806459,7190.09,7190.05
1577836806474,7190.43,7190.05
1577836806575,7190.39,7190.05
...,...,...
1577837711136,7178.32,7178.31
1577837711173,7178.32,7178.31
1577837712988,7178.32,7178.31
1577837713474,7178.32,7178.18


In [9]:
print('assetA.asks.mean():',assetA.asks.mean())
print('assetA.bids.mean():',assetA.bids.mean())

assetA.asks.mean(): 7180.519816176471
assetA.bids.mean(): 7180.074784313725


In [10]:
assetB = df.assetB.apply([asks, bids])

,asks,bids
1577836803078,7170.25,7169.5
1577836806371,7170.25,7169.5
1577836806459,7170.50,7169.5
1577836806474,7170.50,7169.5
1577836806575,7170.50,7169.5
...,...,...
1577837711136,7162.75,7159.0
1577837711173,7162.75,7162.0
1577837712988,7162.75,7161.0
1577837713474,7162.75,7161.0


In [ ]:
print('assetB.asks.mean():',assetB.asks.mean())
print('assetB.bids.mean():',assetB.bids.mean())

In [12]:
# short term exponential moving average
short_ema = assetA.ewm(span=12,adjust=False).mean()

,asks,bids
1577836803078,7189.000000,7188.890000
1577836806371,7189.167692,7189.068462
1577836806459,7189.309586,7189.219467
1577836806474,7189.481957,7189.347242
1577836806575,7189.621656,7189.455358
...,...,...
1577837711136,7178.268262,7178.217060
1577837711173,7178.276222,7178.231358
1577837712988,7178.282957,7178.243457
1577837713474,7178.288656,7178.233694


In [ ]:
plot(short_ema, 'Short Moving Average')

<ipython-input-7-5f23de08327d>:6: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  axes = plt.axes()


In [ ]:
# long term exponential moving average
long_ema = assetA.ewm(span=26,adjust=False).mean()
plot(long_ema,'Long Moving Average')

# getmeths(axes.patch, only='regular')

In [ ]:
#MACD line
macd = short_ema - long_ema
plot(macd, title = 'MACD')

In [ ]:
# signal line
signal = macd.ewm(span=9, adjust=False).mean()
plot(signal, title = 'Signal')

In [ ]:
# plot
plotmany(df.index,[
    (macd, dict(label = 'MACD', color='orange')),
    (signal, dict(label = 'Signal',color='lightblue'))
    ])



In [ ]:
# plot
plotmany(df.index,[
    (macd, dict(label = 'MACD', color='orange')),
    (signal, dict(label = 'Signal',color='lightblue'))
    ])



In [ ]:
# plot
plotmany(df.index,[
    (macd, dict(label = 'MACD', color='orange')),
    (signal, dict(label = 'Signal',color='lightblue'))
    ])



In [ ]:
# plot
plotmany(df.index,[
    (macd, dict(label = 'MACD', color='orange')),
    (signal, dict(label = 'Signal',color='lightblue'))
    ])



In [ ]:
# plot
plotmany(df.index,[
    (macd, dict(label = 'MACD', color='orange')),
    (signal, dict(label = 'Signal',color='lightblue'))
    ])



In [ ]:
# plot
plotmany(df.index,[
    (macd, dict(label = 'MACD', color='orange')),
    (signal, dict(label = 'Signal',color='lightblue'))
    ])



In [ ]:
# plot
plotmany(df.index,[
    (macd, dict(label = 'MACD', color='orange')),
    (signal, dict(label = 'Signal',color='lightblue'))
    ])



In [ ]:
# plot
plotmany(df.index,[
    (macd, dict(label = 'MACD', color='orange')),
    (signal, dict(label = 'Signal',color='lightblue'))
    ])



In [ ]:
# plot
plotmany(df.index,[
    (macd, dict(label = 'MACD', color='orange')),
    (signal, dict(label = 'Signal',color='lightblue'))
    ])



In [ ]:
# plot
plotmany(df.index,[
    (macd, dict(label = 'MACD', color='orange')),
    (signal, dict(label = 'Signal',color='lightblue'))
    ])

